In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import tensorflow as tf
import numpy as np

# Set the seed for random operations. 
# This let our experiments to be reproducible. 
tf.random.set_seed(1234)  

## Dataset

### tf.data.Dataset.range

In [ ]:
# Create a Dataset of sequential numbers
# --------------------------------------
print("Dataset.range examples:")
print("-----------------------")

range_dataset = tf.data.Dataset.range(0, 20, 1)

print("\n1. Dataset")
for el in range_dataset:
    print(el)

# Divide in batches
bs = 3
range_dataset = tf.data.Dataset.range(0, 20, 1).batch(bs, drop_remainder=False)

print("\n2. Dataset + batch")
for el in range_dataset:
    print(el)

# Apply a transformation to each element
def map_fn(x):
    return x**2

range_dataset = tf.data.Dataset.range(0, 20, 1).batch(bs, drop_remainder=False).map(map_fn)

print("\n3. Dataset + batch + map")
for el in range_dataset:
    print(el)

# Filter dataset based on a condition
def filter_fn(x):
    return tf.equal(tf.math.mod(x, 2), 0)

range_dataset = tf.data.Dataset.range(0, 20, 1).filter(filter_fn)

print("\n4. Dataset + filter")
for el in range_dataset:
    print(el)

# Random shuffling
range_dataset = tf.data.Dataset.range(0, 20, 1).shuffle(
    buffer_size=20, reshuffle_each_iteration=False, seed=1234).batch(bs)

print("\n5. Dataset + shuffle + batch")
for el in range_dataset:
    print(el)



### tf.data.Dataset.from_tensors

In [ ]:
# Create Dataset as unique element
# --------------------------------
from_tensors_dataset = tf.data.Dataset.from_tensors([1, 2, 3, 4, 5, 6, 7, 8, 9])

print("Dataset.from_tensors example:")
print("-----------------------------")
for el in from_tensors_dataset:
    print(el)

### tf.data.Dataset.from_tensor_slices

In [ ]:
# Create a Dataset of slices
# --------------------------

# All the elements must have the same size in first dimension (axis 0)
from_tensor_slices_dataset = tf.data.Dataset.from_tensor_slices(
    (np.random.uniform(size=[10, 2, 2]), np.random.randint(10, size=[10])))

print("Dataset.from_tensor_slices example:")
print("-----------------------------")
for el in from_tensor_slices_dataset:
    print(el)

### tf.data.Dataset.zip

In [ ]:
# Combine multiple datasets
# -------------------------
x = tf.data.Dataset.from_tensor_slices(np.random.uniform(size=10))
y = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6, 7, 8, 9])

zipped = tf.data.Dataset.zip((x, y))

print("Dataset.from_tensors example:")
print("-----------------------------")
for el in zipped:
    print(el)

In [ ]:
# Iterate over range dataset
# --------------------------

# for a in b
for el in zipped:
    print(el)
    
print('\n')
    
# for a in enumerate(b)
for el_idx, el in enumerate(zipped):
    print(el)
    
print('\n')
    
# get iterator
iterator = iter(zipped)
print(next(iterator))

# Example: Fashion MNIST - Multi-class classification
## Dataset

In [ ]:
# Load built-in dataset
# ---------------------
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
x_train.shape
y_train.shape

In [ ]:
# Split in training and validation sets
# e.g., 50000 samples for training and 10000 samples for validation

x_valid = x_train[50000:, ...] 
y_valid = y_train[50000:, ...] 

x_train = x_train[:50000, ...]
y_train = y_train[:50000, ...]

In [ ]:
# Create Training Dataset object
# ------------------------------
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))

# Shuffle
train_dataset = train_dataset.shuffle(buffer_size=x_train.shape[0])

# Normalize images
def normalize_img(x_, y_):
    return tf.cast(x_, tf.float32) / 255., y_

train_dataset = train_dataset.map(normalize_img)

# 1-hot encoding <- for categorical cross entropy
def to_categorical(x_, y_):
    return x_, tf.one_hot(y_, depth=10)

train_dataset = train_dataset.map(to_categorical)

# Divide in batches
bs = 32
train_dataset = train_dataset.batch(bs)

# Repeat
# Without calling the repeat function the dataset 
# will be empty after consuming all the images
train_dataset = train_dataset.repeat()

In [ ]:
# Create Validation Dataset  
# -----------------------
valid_dataset = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))

# Normalize images
valid_dataset = valid_dataset.map(normalize_img)

# 1-hot encoding
valid_dataset = valid_dataset.map(to_categorical)

# Divide in batches
valid_dataset = valid_dataset.batch(1)

# Repeat
valid_dataset = valid_dataset.repeat()

In [ ]:
# Create Test Dataset
# -------------------
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

test_dataset = test_dataset.map(normalize_img)

test_dataset = test_dataset.map(to_categorical)

test_dataset = test_dataset.batch(1)

In [ ]:
# Check that is everything is ok..

iterator = iter(train_dataset)
sample, target = next(iterator)

# Just for visualization purpouses
sample = sample[0, ...]  # select first image in the batch
sample = sample * 255  # denormalize

from PIL import Image
img = Image.fromarray(np.uint8(sample))
img = img.resize([128, 128])
img

target[0]  # select corresponding target

## Model

In [ ]:
# Fashion MNIST classification
# ----------------------------

# x: 28x28
# y: 10 classes

# Create Model
# ------------
# e.g. in: 28x28 -> h: 10 units -> out: 10 units (number of classes) 

# Define Input keras tensor
x = tf.keras.Input(shape=[28, 28])

# Define intermediate hidden layers and chain
flatten = tf.keras.layers.Flatten()(x)
h = tf.keras.layers.Dense(units=10, activation=tf.keras.activations.sigmoid)(flatten)

# Define output layer and chain
out = tf.keras.layers.Dense(units=10, activation=tf.keras.activations.softmax)(h)

# Create Model instance defining inputs and outputs
model = tf.keras.Model(inputs=x, outputs=out)

In [ ]:
# Visualize created model as a table
model.summary()

# Visualize initialized weights
model.weights

In [ ]:
# Equivalent formulation
# ----------------------

# Create model with sequential 
# (uncomment to run)
# seq_model = tf.keras.Sequential()
# seq_model.add(tf.keras.layers.Flatten(input_shape=(28, 28))) # or as a list
# seq_model.add(tf.keras.layers.Dense(units=10, activation=tf.keras.activations.sigmoid))
# seq_model.add(tf.keras.layers.Dense(units=10, activation=tf.keras.activations.softmax))

In [ ]:
# seq_model.summary()
# seq_model.weights

## Prepare the model for training

In [ ]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

## Training

In [ ]:
model.fit(x=train_dataset,  # you can give directly numpy arrays x_train
          y=None,   # if x is a Dataset y has to be None, y_train otherwise
          epochs=10, 
          steps_per_epoch=int(np.ceil(x_train.shape[0] / bs)),  # how many batches per epoch
          validation_data=valid_dataset,  # give a validation Dataset if you created it manually, otherwise you can use 'validation_split' for automatic split
          validation_steps=10000)  # number of batches in validation set

## Training with callbacks

In [ ]:
import os
from datetime import datetime

cwd = os.getcwd()

exps_dir = os.path.join(cwd, 'experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

exp_dir = os.path.join(exps_dir, 'exp_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'ckpt_{epoch:02d}'), 
                                                   save_weights_only=True)  # False to save the model directly
# ----------------

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             histogram_freq=1)  # if 1 shows weights histograms
# ---------------------------------

model.fit(x=train_dataset,
          epochs=10,  #### set repeat in training dataset
          steps_per_epoch=int(np.ceil(x_train.shape[0] / bs)),
          validation_data=valid_dataset,
          validation_steps=10000, 
          callbacks=[ckpt_callback, tb_callback])

# How to visualize Tensorboard

# 1. tensorboard --logdir EXPERIMENTS_DIR --port PORT     <- from terminal
# 2. localhost:PORT   <- in your browser

## Test model

In [ ]:
# Let's try a different way to give data to model 
# using directly the NumPy arrays

# model.load_weights('/path/to/checkpoint')  # use this if you want to restore saved model

eval_out = model.evaluate(x=x_test / 255.,
                          y=tf.keras.utils.to_categorical(y_test),
                          verbose=0)

eval_out

## Compute prediction

In [ ]:
# Compute output given x

shoe_img = Image.open('shoe.png').convert('L')

shoe_arr = np.expand_dims(np.array(shoe_img), 0)

out_softmax = model.predict(x=shoe_arr / 255.)

out_softmax  # is already a probability distribution (softmax)

In [ ]:

out_softmax = tf.keras.activations.softmax(tf.convert_to_tensor(out_softmax))
out_softmax

In [ ]:
# Get predicted class as the index corresponding to the maximum value in the vector probability
predicted_class = tf.argmax(out_softmax, 1)
predicted_class